In [ ]:
# Importing libraries

from gurobipy import *

import numpy as np

import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
m = Model();

In [ ]:
# Static parameters
number_of_gates = 9
number_of_carts = 4

# Process parameters in seconds
unload_time = 300
load_time = 300
travel_time = 120

## Defining sets

### Basic sets

$\begin{gather}
I = 0,1, \dots, (number\;of\;gates)\\
J = 0,1, \dots, ((number\;of\;gates\;\times\;2)\;-1)\\
K = 0,1, \dots, ((number\;of\;gates\;\times\;2)\;-2)\\
\end{gather}$

In [ ]:
set_I = [I for I in range(0,number_of_gates+1)] # I ---> 0 to 9
set_J = [J for J in range(0,number_of_gates*2)] # J ---> 0 to 17
set_K = [K for K in range(0,number_of_gates*2-1)] # K ---> 0 to 16

### Subsidiary sets

$\begin{gather}
i = 1,2 \dots, (number\;of\;gates)\\
j = 1,2 \dots, ((number\;of\;gates\;\times\;2)\;-1)\\
k = 1,2 \dots, ((number\;of\;gates\;\times\;2)\;-2)\\
\end{gather}$

In [ ]:
set_i = [i for i in range(1,number_of_gates+1)] # i --> 1 to 9
set_j = [j for j in range(1,number_of_gates*2)] # j ---> 1 to 17
set_k = [k for k in range(1,number_of_gates*2-1)] # k ---> 1 to 16

## Defining variables

$ x_{J,I} $: $\begin{gather} 1\;If\;the\;rampie\;delivers\;baggage\;belonging\;to\;gate\;I\;in\;J^{th}\;attempt.\;Else\;0 \end{gather}$

$ t_{K,I} $: $\begin{gather} 1\;If\;the\;rampie\;picks\;baggage\;belonging\;to\;gate\;I\;in\;J^{th}\;attempt.\;Else\;0 \end{gather}$

In [ ]:
x = m.addVars(set_J, set_I, name="x", lb = +0.0, vtype=GRB.BINARY)
t = m.addVars(set_K, set_I, name="t", lb = +0.0, vtype=GRB.BINARY)

## Defining constraints

$$ \begin{aligned}
\quad&\sum_{K=0}^{(number\;of\;gates\;\times\;2)-2}t_{K,0} = (number\;of\;gates\;\times\;2)-1\\
\forall \; i \in Set\;i \quad&\sum_{K=0}^{(number\;of\;gates\;\times\;2)-2}t_{K,i} = 1\\
\forall \; K \in Set\;K \quad&\sum_{I=0}^{(number\;of\;gates)}t_{K,I} = (number\;of\;carts)+1\\
\end{aligned} $$

In [ ]:
m.addConstrs(      (    sum(t[K,a] for K in set_K) == number_of_gates*2-1 for a in [0]   )         ,"con1.1");
m.addConstrs(      (    sum(t[K,i] for K in set_K) == 1 for i in set_i    )         ,"con1.2");
m.addConstrs(      (    sum(t[K,I] for I in set_I) <= number_of_carts+1 for K in set_K   )         ,"con1.3");

-------------------------------------------------------------------------------------------------------------------------------

$$ \begin{aligned}
\forall \; i \in Set\;i \quad&\sum_{J=0}^{(number\;of\;gates\;\times\;2)-1} x_{J,i} = 1\\
\forall \; J \in Set\;J \quad&\sum_{I=0}^{(number\;of\;gates)} x_{J,I} = 1\\
\end{aligned} $$

In [ ]:
m.addConstrs(      (    sum(x[J,i] for J in set_J) == 1 for i in set_i   )         ,"con2.1");
m.addConstrs(      (    sum(x[J,I] for I in set_I) <= 1 for J in set_J   )         ,"con2.2");

-------------------------------------------------------------------------------------------------------------------------------

$$ \begin{aligned}
\forall \; j \in Set\;j\;\;\;\forall \; i \in Set\;i\quad&x_{j,i}\;-\;\sum_{K=0}^{(j^{th}\;element\;in\;Set\;K)} t_{K,i} \leqslant 0\\
\end{aligned} $$

In [ ]:
m.addConstrs(      (    x[j,i] - sum(t[K,i] for K in set_K[:j]) <= 0 for i in set_i for j in set_j   )         ,"con3.1");

-----------------------------------------------------------------------------------------------------------------------------

$$ \begin{aligned}
\forall \; K \in Set\;K\quad&x_{j,i}\times (Number\;of\;gates)\;-\;\sum_{i=1}^{(Number\;of\;gates)} t_{K,i} \geqslant 0\\
\forall \; K \in Set\;K\quad&(Number\;of\;gates)-1\geqslant x_{j,i}\times (Number\;of\;gates)\;-\;\sum_{i=1}^{(Number\;of\;gates)} t_{K,i} \\
\end{aligned} $$

In [ ]:
m.addConstrs(      (     number_of_carts*x[K,0] - sum(t[K,i] for i in set_i) >= 0 for K in set_K   )         ,"con4.1");

# m.addConstrs(      (     number_of_carts > number_of_carts*x[K,0] - sum(t[K,i] for i in set_i) for K in set_K   )         ,"con4.2")
# The above constraint gave me "No implementation error" for some reason of using > instead of >=

m.addConstrs(      (     number_of_carts-1 >= number_of_carts*x[K,0] - sum(t[K,i] for i in set_i) for K in set_K   )         ,"con4.2");

-------------------------------------------------------------------------------------------------------------------------------

$$ \begin{aligned}
\forall \; k \in Set\;k \quad&\sum_{K=0}^{((k+1)^{th}\;element\;in\;set\;K)}\sum_{i=1}^{(Number\;of\;gates)} t_{K,i} - \sum_{K=0}^{(k^{th}\;element\;in\;set\;K)}\sum_{i=1}^{(Number\;of\;gates)} x_{K,i} \leqslant (Number\;of\;carts)\\
\end{aligned} $$

In [ ]:
m.addConstrs(      (    sum(t[K,i] for i in set_i for K in set_K[:k+1]) - sum(x[K,i] for i in set_i for K in set_K[:k]) <= number_of_carts for k in set_k   )         ,"con4.3");

-------------------------------------------------------------------------------------------------------------------------------

$$ \begin{aligned}
\forall \; J \in (First\;element\;in\;Set\;J)\;\forall \; I \in (First\;element\;in\;Set\;I) \quad&x_{J,I} = 1\\
\forall \; I \in (First\;element\;in\;Set\;I) \quad&x_{1,I} = 1\\
\end{aligned} $$

In [ ]:
m.addConstrs(      (x[J,I] ==  1 for J in set_J[:1] for I in set_I[:1]  )     ,"con5.1");
m.addConstrs(      (x[1,I] ==  0 for I in set_I[:1]  )     ,"con5.2");

-------------------------------------------------------------------------------------------------------------------------------

$$ \begin{aligned}
\forall \; K \in Set\;K\;\forall \; j \in (K^{th}\;element\;till\;end\;of\;Set\;j) \quad&\sum_{K=0}^{(number\;of\;gates\;\times\;2)-2} x_{K,I}\geqslant\\ \sum_{I=0}^{(number\;of\;gates)} x_{j,I}
\end{aligned} $$

In [ ]:
for K in set_K:
    for j in set_j[K:]:
        m.addConstr((     sum(x[K,I] for I in set_I) >= sum(x[j,I] for I in set_I)     ))

-------------------------------------------------------------------------------------------------------------------------------

$$ \begin{aligned}
\forall \; (j,K) \in (Set\;j,\;Set\;K)\quad& x_{K,0} + x_{j,0}\leqslant 1\\
\end{aligned} $$

In [ ]:
for j,K in zip(set_j,set_K):
    m.addConstr(      (x[K,0] + x[j,0] <=  1  )     )

## Objective function (Dummy)

In [ ]:
m.setObjective(0, GRB.MAXIMIZE)

## Setting solver parameters

In [ ]:
# n=18192
n=15
m.setParam('PoolSolutions',n) # 10 by default
m.setParam('PoolSearchMode',2) # 0 by default
# m.setParam('IntegralityFocus',1) # This parameter will make sure during MIPs 1 cannot shows 1.00009 or 0.9999. It strictly bounds to integer values only. Seemingly it does not work.

m.update() # Not sure when exactly to use it
m.optimize()

Set parameter PoolSolutions to value 15
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 429 rows, 350 columns and 8240 nonzeros
Model fingerprint: 0xdd03bbda
Variable types: 0 continuous, 350 integer (350 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 91 rows and 36 columns
Presolve time: 0.03s
Presolved: 338 rows, 314 columns, 5618 nonzeros
Variable types: 0 continuous, 314 integer (313 binary)

Root relaxation: objective -0.000000e+00, 168 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0          -    

# Statistics

In [ ]:
# m.computeIIS() # Dont know what exactly this is. It works only on infeasible solutions

# m.write('C:\\Users\\sabbe\\Desktop\\UPitt\\Optimization\\United_airlines_bagrunner_optimization\\model.lp')
# m.write('C:\\Users\\sabbe\\Desktop\\UPitt\\Optimization\\United_airlines_bagrunner_optimization\\model.mps')
# print (m.display())

In [ ]:
# m.printQuality()
# m.printStats()

# Answer check

In [ ]:
#m.setParam('SolutionNumber',12250)
#m.setParam('SolutionNumber',1)

for I in set_I:
    globals()['sol%s'%I]=[]
    for K in set_K:
        globals()['sol%s'%I].append(t[K,I].xn)
    print(globals()['sol%s'%I])
    

print('---------------------------------------------------------------------')

for I in set_I:
    globals()['soll%s'%I]=[]
    for J in set_J:
        globals()['soll%s'%I].append(x[J,I].Xn)
    print(globals()['soll%s'%I])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---------------------------------------------------------------------
[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,

## Solutions in a dataset

In [ ]:
sol_pool = pd.read_excel('C:\\Users\\sabbe\\Desktop\\UPitt\\Optimization\\United_airlines_bagrunner_optimization\\Solution_pool_template.xlsx',sheet_name = "Sheet1")

In [ ]:
#nn = 18192
n
for r in range(0,n):
    m.setParam('SolutionNumber',r) # 0 by default
    row = r
    for K in set_K:
        string=""
        for i in set_i:
            if t[K,i].xn >= 0.5:
                string = string + str(i) + "p,"
        if string != "":
            sol_pool.at[row, 'attempt ' + str(K)] = string[:len(string)-1]
        else:
            sol_pool.at[row, 'attempt ' + str(K)] = str(int(sum(round(x[K,ii].xn)*ii for ii in set_i))) + 'd'
    la = number_of_gates*2-1
    sol_pool.at[row, 'attempt ' + str(la)] = str(int(sum(round(x[K,ii].xn)*ii for ii in set_i))) + 'd'

In [ ]:
def count(s):
    c=0
    for i in range(0,len(s)):
        if s[i] == 'p':
            c = c+1
    return c

for l in range(0,len(sol_pool)):
    total_time = 0
    globals()['dict_%s'%l] = {}
    
    for K in set_K:
        if sol_pool['attempt '+str(K)].iloc[l] == "0d":
            break
        if K == 0:
            total_time = count(sol_pool['attempt '+str(K)].iloc[l])*load_time
            globals()['dict_%s'%l].update({sol_pool['attempt '+str(K)].iloc[l]:total_time})
        else:
            if "d" in sol_pool['attempt '+str(K)].iloc[l]:
                if "p" in sol_pool['attempt '+str(K-1)].iloc[l]:
                    total_time = total_time + unload_time + travel_time * int(sol_pool['attempt '+str(K)].iloc[l][:-1])
                    globals()['dict_%s'%l].update({sol_pool['attempt '+str(K)].iloc[l]:total_time})
                else:
                    total_time = total_time + unload_time + travel_time * abs( int(sol_pool['attempt '+str(K-1)].iloc[l][:-1]) - int(sol_pool['attempt '+str(K)].iloc[l][:-1])  )
                    globals()['dict_%s'%l].update({sol_pool['attempt '+str(K)].iloc[l]:total_time})
            else:
                total_time = total_time + load_time*(count(sol_pool['attempt '+str(K)].iloc[l])) + travel_time * int(sol_pool['attempt '+str(K-1)].iloc[l][:-1])
                globals()['dict_%s'%l].update({sol_pool['attempt '+str(K)].iloc[l]:total_time})
                
    sol_pool.at[l, 'total_time'] =  list(globals()['dict_%s'%l].values())[-1]/3600

In [ ]:
sol_pool['sequence'] = str('')
for i in range(0,len(sol_pool)):
    for j in range(0,len(sol_pool.columns)-2):
        sol_pool['sequence'].iloc[i] = str(sol_pool['sequence'].iloc[i])+str(sol_pool.iloc[i,j])

In [ ]:
sol_pool

,attempt 0,attempt 1,attempt 2,attempt 3,attempt 4,attempt 5,attempt 6,attempt 7,attempt 8,attempt 9,attempt 10,attempt 11,attempt 12,attempt 13,attempt 14,attempt 15,attempt 16,attempt 17,total_time,sequence
0,"2p,5p,8p,9p",2d,8d,"6p,7p",9d,7d,5d,"1p,3p",3d,1d,6d,4p,4d,0d,0d,0d,0d,0d,3.300000,"2p,5p,8p,9p2d8d6p,7p9d7d5d1p,3p3d1d6d4p4d0d0d0..."
1,"1p,4p,8p",1d,5p,4d,"3p,9p",8d,9d,2p,3d,5d,2d,6p,6d,7p,7d,0d,0d,0d,3.400000,"1p,4p,8p1d5p4d3p,9p8d9d2p3d5d2d6p6d7p7d0d0d0d"
2,"5p,7p,9p",9d,7d,"3p,4p,6p",6d,4d,3d,"1p,2p,8p",1d,2d,8d,5d,0d,0d,0d,0d,0d,0d,2.866667,"5p,7p,9p9d7d3p,4p,6p6d4d3d1p,2p,8p1d2d8d5d0d0d..."
3,"2p,8p",8d,"3p,5p",2d,4p,4d,5d,1p,3d,7p,7d,"6p,9p",6d,9d,1d,0d,0d,0d,3.733333,"2p,8p8d3p,5p2d4p4d5d1p3d7p7d6p,9p6d9d1d0d0d0d"
4,"1p,4p,7p,8p",1d,4d,7d,"3p,9p",8d,9d,"2p,5p",3d,5d,2d,6p,6d,0d,0d,0d,0d,0d,3.100000,"1p,4p,7p,8p1d4d7d3p,9p8d9d2p,5p3d5d2d6p6d0d0d0..."
5,"1p,2p,7p",1d,4p,2d,6p,6d,7d,"3p,9p",9d,4d,5p,5d,3d,8p,8d,0d,0d,0d,3.366667,"1p,2p,7p1d4p2d6p6d7d3p,9p9d4d5p5d3d8p8d0d0d0d"
6,"1p,4p,8p",8d,5p,4d,"3p,9p",1d,9d,2p,3d,5d,2d,6p,6d,7p,7d,0d,0d,0d,3.866667,"1p,4p,8p8d5p4d3p,9p1d9d2p3d5d2d6p6d7p7d0d0d0d"
7,"1p,4p,5p,8p",1d,5d,8d,"6p,7p,9p",9d,6d,"2p,3p",7d,2d,3d,4d,0d,0d,0d,0d,0d,0d,3.100000,"1p,4p,5p,8p1d5d8d6p,7p,9p9d6d2p,3p7d2d3d4d0d0d..."
8,"1p,3p,4p,9p",4d,5p,9d,8p,8d,3d,2p,2d,6p,5d,1d,6d,7p,7d,0d,0d,0d,3.933333,"1p,3p,4p,9p4d5p9d8p8d3d2p2d6p5d1d6d7p7d0d0d0d"
9,"1p,4p,6p,8p",6d,5p,4d,3p,8d,3d,2p,2d,9p,5d,1d,9d,7p,7d,0d,0d,0d,3.933333,"1p,4p,6p,8p6d5p4d3p8d3d2p2d9p5d1d9d7p7d0d0d0d"


## Checking for duplicates

In [ ]:
for i in range(0,len(sol_pool)):
    if sol_pool["sequence"].duplicated()[i] == True:
        print(sol_pool["sequence"].duplicated()[i])

## Solution pool export
### Block code with error

In [ ]:
print(Code intentionally blocked here)

writer = pd.ExcelWriter('Solution.xlsx')
sol_pool.to_excel(writer, 'Solution', na_rep='NA')
writer.save()

SyntaxError: invalid syntax (Temp/ipykernel_9484/3641060880.py, line 1)